In [2]:
import pandas as pd
import numpy as np
import os
import re

In [4]:
collegeboard = pd.DataFrame()
for year in range(18, 23):  # range is exclusive at the end
    filename = f"/Users/adamcartwright/ncerdc/Student Data/Testing Data/collegeboard{year}pub.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values

        
        # Append to the main DataFrame
        collegeboard = pd.concat([collegeboard, df], ignore_index=True)
        print("{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")
        
def clean_bytes(val):
    if isinstance(val, bytes):
        return str(val)[2:-1]  # str(b'xyz') => "b'xyz'" → "xyz"
    return val
for col in collegeboard.columns:
    collegeboard[col] = collegeboard[col].apply(clean_bytes)
    print(col)

cols_to_drop = [col for col in collegeboard.columns if col.endswith('gr')]
collegeboard = collegeboard.drop(columns=cols_to_drop)

collegeboard = collegeboard.dropna(subset=['mastid'])
        
collegeboard.to_csv("/Users/adamcartwright/NCERDC-MODEL/data/collegeboard_master.csv",index=False)

{year} done
{year} done
{year} done
{year} done
{year} done
birthdt
sex
grad_date
ethnic
blang
instname
psat_8_9_test_dt
psat_8_9_total
psat_8_9_ebrw
psat_8_9_math
psat_8_9_tsr
psat_8_9_tsw
psat_8_9_tsm
psat_8_9_sub_evid
psat_8_9_sub_cont
psat_8_9_sub_expr
psat_8_9_sub_conv
psat_8_9_sub_alge
psat_8_9_sub_prob
psat_8_9_ctsh
psat_8_9_ctss
psat_nmsqt_total
psat_nmsqt_ebrw
psat_nmsqt_math
psat_nmsqt_tsr
psat_nmsqt_tsw
psat_nmsqt_tsm
psat_nmsqt_sub_evid
psat_nmsqt_sub_cont
psat_nmsqt_sub_expr
psat_nmsqt_sub_conv
psat_nmsqt_sub_alge
psat_nmsqt_sub_prob
psat_nmsqt_sub_advm
psat_nmsqt_ctsh
psat_nmsqt_ctss
psat_10_test_dt
psat_10_total
psat_10_ebrw
psat_10_math
psat_10_tsr
psat_10_tsw
psat_10_tsm
psat_10_sub_evid
psat_10_sub_cont
psat_10_sub_expr
psat_10_sub_conv
psat_10_sub_alge
psat_10_sub_prob
psat_10_sub_advm
psat_10_ctsh
psat_10_ctss
sat_asmt_dt_mr
sat_total_score_mr
sat_erw_score_mr
sat_math_score_mr
sat_er_mr
sat_ea_mr
sat_ew_mr
sat_tsr_mr
sat_tsw_mr
sat_tsm_mr
sat_sub_evid_mr
sat_sub_co